# Jupyter Notebook zu der Bachelor Thesis von Christian Albrecht "Identifikation der Teilnehmeranzahl in schweizerdeutschen Telefonaufzeichnungen

Laden der beiden Test Aufzeichnungen 1270 und mytest1.

In [ ]:
from pyannote.audio.features import RawAudio
from IPython.display import Audio
arm_file = {'uri': 'd1270', 'audio': '../99_DATASETs/ARCHIMOB/ARCHIMOB_DATA/d1270.wav'}
my_file = {'uri': 'mytest1', 'audio': '../99_DATASETs/SINGLE_FILE/mytest1.wav'}


In [ ]:
# Laden der Grundwahrheiten:
from pyannote.database.util import load_rttm
groundtruth_arm = load_rttm('../99_DATASETs/ARCHIMOB/ARCHIMOB_META/1270_test.rttm')[arm_file['uri']]
groundtruth_my = load_rttm('../99_DATASETs/SINGLE_FILE/mytest1.rttm')[my_file['uri']]

Die folgenden Befehle können benutzt werden um in den Grafiken nur ein Bereicht anzuzeigen.

In [ ]:
from pyannote.core import Segment, notebook
# make notebook visualization zoom on 0 < t < 150s time range
EXCERPT = Segment(0, 500)
notebook.crop = EXCERPT

In [ ]:
groundtruth_arm

In [ ]:
groundtruth_my

# Test von eigenen Audio Dateien

Damit eigene Audiodateien getestet werden können 
__Ersetzen von arm_file mit my_file im restlichen Notebook !!!__ 

Wenn es kein Groundtruth File zu dem eigenen File gibt kann diese natrürlich auch nicht angezeigt werden. 

# Selbst Trainiertes Model auf ArchiMob Datensatz

In [ ]:
sad_scores = "../97_PYANNOTE/pyannote-audio/tutorials/finetune/sad/train/ARM.SpeakerDiarization.ArchiMob.train/weights/0010.pt"
scd_scores = "../97_PYANNOTE/pyannote-audio/tutorials/finetune/scd/train/ARM.SpeakerDiarization.ArchiMob.train/weights/0028.pt"
emb_scores = "../97_PYANNOTE/pyannote-audio/tutorials/finetune/emb/train/ARM.SpeakerDiarization.ArchiMob.train/weights/0016.pt"
method = "affinity_propagation"

In [ ]:
from pyannote.audio.pipeline.speaker_diarization import SpeakerDiarization

In [ ]:
pipeline_self = SpeakerDiarization(
    sad_scores = {sad_scores: {'step': 0.1}},
    scd_scores = {scd_scores: {'step': 0.1}},
    embedding = {emb_scores: {'step': 0.1}},
    method=method)

In [ ]:
params_file = "../97_PYANNOTE/pyannote-audio/tutorials/pipelines/speaker_diarization/train/ARM.SpeakerDiarization.ArchiMob.development/params.yml"

In [ ]:
pipeline_self.load_params(params_file)

In [ ]:
%%time
diarization_self_arm = pipeline_self(arm_file)
diarization_self_my = pipeline_self(my_file)

In [ ]:
diarization_self_arm

In [ ]:
diarization_self_my

# Vergleiche Groundtruth / Diarization ArchiMob

In [ ]:
from pyannote.metrics.diarization import DiarizationErrorRate
metric_self = DiarizationErrorRate()
der_self_arm = metric_self(groundtruth_arm, diarization_self_arm)
der_self_my = metric_self(groundtruth_my, diarization_self_my)

In [ ]:
print(f'diarization error rate self archimob_arm = {100 * der_self_arm:.1f}%')

In [ ]:
print(f'diarization error rate self archimob_my = {100 * der_self_my:.1f}%')

# Pre-trainiertes Model auf AMI Datensatz

In [ ]:
%%time
import torch
pipeline_pretrain_ami = torch.hub.load('pyannote/pyannote-audio', 'dia_ami')
diarization_pretrain_ami_arm = pipeline_pretrain_ami(arm_file)
diarization_pretrain_ami_my = pipeline_pretrain_ami(my_file)

In [ ]:
diarization_pretrain_ami_arm

In [ ]:
diarization_pretrain_ami_my

# Vergleich Groundtruth / Diarization AMI

In [ ]:
from pyannote.metrics.diarization import DiarizationErrorRate
metric_pretrain_ami = DiarizationErrorRate()
der_pretrain_ami_arm = metric_pretrain_ami(groundtruth_arm, diarization_pretrain_ami_arm)
der_pretrain_ami_my = metric_pretrain_ami(groundtruth_my, diarization_pretrain_ami_my)

In [ ]:
print(f'diarization error rate pretrain ami_arm = {100 * der_pretrain_ami_arm:.1f}%')

In [ ]:
print(f'diarization error rate pretrain ami_my = {100 * der_pretrain_ami_my:.1f}%')

# Pre-trainiertes Model auf DIHARD Datensatz

In [ ]:
%%time
import torch
pipeline_pretrain_dihard = torch.hub.load('pyannote/pyannote-audio', 'dia_dihard')
diarization_pretrain_dihard_arm = pipeline_pretrain_dihard(arm_file)
diarization_pretrain_dihard_my = pipeline_pretrain_dihard(my_file)

In [ ]:
diarization_pretrain_dihard_arm

In [ ]:
diarization_pretrain_dihard_my

In [ ]:
from pyannote.metrics.diarization import DiarizationErrorRate
metric_pretrain_dihard = DiarizationErrorRate()
der_pretrain_dihard_arm = metric_pretrain_dihard(groundtruth_arm, diarization_pretrain_dihard_arm)
der_pretrain_dihard_my = metric_pretrain_dihard(groundtruth_my, diarization_pretrain_dihard_my)

In [ ]:
print(f'diarization error rate pretrain dihard_arm = {100 * der_pretrain_dihard_arm:.1f}%')

In [ ]:
print(f'diarization error rate pretrain dihard_my = {100 * der_pretrain_dihard_my:.1f}%')

# Vergleich ArchiMob / AMI 

In [ ]:
print(f'diarization error rate self archimob_arm = {100 * der_self_arm:.1f}%')
print(f'diarization error rate self archimob_my = {100 * der_self_my:.1f}%')
print(f'-------')
print(f'diarization error rate pretrain ami_arm = {100 * der_pretrain_ami_arm:.1f}%')
print(f'diarization error rate pretrain ami_my = {100 * der_pretrain_ami_my:.1f}%')
print(f'-------')
print(f'diarization error rate pretrain dihard_arm = {100 * der_pretrain_dihard_arm:.1f}%')
print(f'diarization error rate pretrain dihard_my = {100 * der_pretrain_dihard_my:.1f}%')

# ARM - SELF

In [ ]:
groundtruth_arm

In [ ]:
diarization_self_arm

In [ ]:
print(f'diarization error rate self archimob_arm = {100 * der_self_arm:.1f}%')

# ------------------------------------------------

# MY - SELF

In [ ]:
groundtruth_my

In [ ]:
diarization_self_my

In [ ]:
print(f'diarization error rate self archimob_my = {100 * der_self_my:.1f}%')

# ----------------------------------------------

# ARM - AMI

In [ ]:
groundtruth_arm

In [ ]:
diarization_pretrain_ami_arm

In [ ]:
print(f'diarization error rate pretrain ami_arm = {100 * der_pretrain_ami_arm:.1f}%')

# MY - AMI

In [ ]:
groundtruth_my

In [ ]:
diarization_pretrain_ami_my

In [ ]:
print(f'diarization error rate pretrain ami_my = {100 * der_pretrain_ami_my:.1f}%')

# ARM - DIHARD

In [ ]:
groundtruth_arm

In [ ]:
diarization_pretrain_dihard_arm

In [ ]:
print(f'diarization error rate pretrain dihard_arm = {100 * der_pretrain_dihard_arm:.1f}%')

# MY - DIHARD

In [ ]:
groundtruth_my

In [ ]:
diarization_pretrain_dihard_my

In [ ]:
print(f'diarization error rate pretrain dihard_my = {100 * der_pretrain_dihard_my:.1f}%')